# Make All Tables for Main Manuscript

Kendra Wyant  
2025-12-27

In [ ]:
#| message: false
#| warning: false

suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(source("https://github.com/jjcurtin/lab_support/blob/main/format_path.R?raw=true"))
library(kableExtra)
library(janitor)

path_shared <- format_path("risk2/data_processed/shared")

options(knitr.kable.NA = '')

## Table 1 participant demographics

Table 1

In [ ]:
study_dates <- read_csv(file.path(path_shared, "study_dates_gps.csv"), 
                        show_col_types = FALSE)

intake <- read_csv(file.path(path_shared, "survey_intake.csv"), 
                   show_col_types = FALSE) |>
  filter(subid %in% study_dates$subid) 


# Calcs to make df for table 1 (demographics and clinical characteristics)
n_total <- nrow(study_dates)

dem_age <-  intake |>
  select(var = age) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100) |> 
  add_row(var = "Age", .before = 1)

dem_gender <-  intake |>
  select(var = gender) |>
  filter(var != "Prefer not to say") |> 
  mutate(var = factor(var, 
                      levels = c("Man",
                                 "Woman",
                                 "Non-binary",
                                 "Not listed above"))) |> 
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100) |> 
  add_row(var = "Gender", .before = 1)

dem_orientation <-  intake |>
  select(var = orientation) |>
  filter(!is.na(var)) |> 
  mutate(var = factor(var, levels = c("Straight, that is, not gay or lesbian",
                                      "Lesbian or gay",
                                      "Bisexual",
                                      "Not sure",
                                      "Not listed above"))) |> 
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100) |> 
  add_row(var = "Orientation", .before = 1)

dem_race <- intake |>
  select(starts_with("race_"), -race_other_text) |> 
  pivot_longer(1:7, names_to = "var") |> 
  mutate(var = case_match(var,
                           "race_ai_an" ~ "American Indian/Alaskan Native",
                           "race_asian" ~ "Asian",
                           "race_black" ~ "Black/African American",
                           "race_nat_hi" ~ "Native Hawaiian/Other Pacific Islander",
                           "race_white" ~ "White/Caucasian",
                           "race_hispanic" ~ "Hispanic, Latino, or Spanish origin",
                           "race_other" ~ "Not listed above"),
         var = factor(var,levels = c("American Indian/Alaskan Native",
                                     "Asian",
                                     "Black/African American",
                                     "Native Hawaiian/Other Pacific Islander",
                                     "White/Caucasian",
                                     "Hispanic, Latino, or Spanish origin",
                                     "Not listed above"))) |> 
  group_by(var) |>
  count(value) |> 
  ungroup() |> 
  filter(value == "yes") |> 
  select(-value) |> 
  mutate(perc = (n / n_total) * 100) |>
  add_row(var = "Race and Ethnicity (select all that apply)", .before = 1)
  

dem_education <- intake |>
  select(var = education) |>
  filter(!is.na(var)) |> 
  mutate(var = factor(var, 
                      levels = c("8th grade or less",
                                 "Some high school, but did not graduate",
                                 "High school graduate or GED",
                                 "Some college or 2-year degree",
                                 "4-year college graduate",
                                 "More than 4-year or advanced degree"))) |> 
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100) |>
  add_row(var = "Education", .before = 1)

dem_employment <- intake |>
  select(var = employment) |>
  filter(!is.na(var)) |> 
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100) |>
  add_row(var = "Employment", .before = 1)

dem_income <- intake |>
  select(var = income) |>
  filter(!is.na(var)) |> 
  mutate(var = factor(var, 
                      levels = c("Less than $25,000",
                                 "$25,000 - $34, 999",
                                 "$35,000 - $49,999",
                                 "$50,000 - $74, 999",
                                 "$75, 000 - $99, 999",
                                 "$100,000 - $149,999",
                                 "$150, 000 - $199,999",
                                 "$200, 000 or more"),
                      labels = c("Less than $25,000",
                                 "$25,000 - $34,999",
                                 "$35,000 - $49,999",
                                 "$50,000 - $74,999",
                                 "$75,000 - $99,999",
                                 "$100,000 - $149,999",
                                 "$150,000 - $199,999",
                                 "$200,000 or more"))) |> 
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100) |>
  add_row(var = "Household Income", .before = 1)


dsm <- intake |>
  summarise(mean = mean(dsm_c, na.rm = TRUE),
            SD = sd(dsm_c, na.rm = TRUE),
            min = min(dsm_c, na.rm = TRUE),
            max = max(dsm_c, na.rm = TRUE)) |>
  mutate(var = "DSM-5 OUD Symptom Count",
        n = as.numeric(""),
        perc = as.numeric("")) |>
  select(var, n, perc, everything())

past_month_opioid <- intake |>
  select(var = whoassist_month_1_opioid) |>
  mutate(var = factor(var, levels = c("no", "yes"),
                      labels = c("No", "Yes"))) |> 
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100) |>
  drop_na() |> 
  add_row(var = "Past Month Opioid Use", .before = 1)

past_month_detox <- intake |>
  select(var = other_tx_2) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100) |>
  drop_na() |> 
  add_row(var = "Past Month Detox or Residential Treatment", .before = 1)

past_month_psych <- intake |>
  select(var = other_tx_1) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100) |>
  drop_na() |> 
  add_row(var = "Past Month Psychiatric Medication", .before = 1)

opioid_preferred <- intake |>
  select(var = whoassist_opioid_3) |>
  mutate(var = factor(var,
                      levels = c("Fentanyl (Sublimaze, Actiq, apache, goodfella, TNT)",
                                 "Heroin (black tar, black pearl, black, china white, dope, white lady, smack, snow, speedball)",
                                 "Prescription opioid not for opioid treatment (OxyContin, Vicodin, Percocet, Opana)",
                                 "Medication for opioid treatment (e.g., Suboxone, Methadone, etc.)"),
                      labels = c("Fentanyl",
                                 "Heroin",
                                 "Prescription opioid not for opioid treatment",
                                 "Medication for opioid treatment"))) |> 
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100) |>
  drop_na() |> 
  add_row(var = "Preferred Opioid", .before = 1)

roa_preferred <- intake |>
  select(var = whoassist_opioid_4) |>
  mutate(var = factor(var, 
                      levels = c("Injection (IV, muscle, under skin)",
                                 "Oral (swallow, chew)",
                                 "Smoke (chase)",
                                 "Sniff or snort",
                                 "Other"),
                      labels = c("Injection",
                                 "Oral",
                                 "Smoke",
                                 "Sniff or snort",
                                 "Other"))) |> 
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100) |>
  drop_na() |> 
  add_row(var = "Preferred Route of Administration", .before = 1)

od <- intake |>
  select(var = whoassist_life_overdose) |>
  mutate(var = factor(var, 
                      levels = c("No",
                                 "1 time",
                                 "2-3 times",
                                 "4-5 times",
                                 "More than 5 times"),
                      labels = c("Never",
                                 "1 time",
                                 "2-3 times",
                                 "4-5 times",
                                 "More than 5 times"))) |> 
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100) |>
  drop_na() |> 
  add_row(var = "Lifetime History of Overdose", .before = 1)


table_dem <- dem_age |> 
  bind_rows(dem_gender) |> 
  bind_rows(dem_orientation) |> 
  bind_rows(dem_race) |> 
  bind_rows(dem_education) |> 
  bind_rows(dem_employment) |> 
  bind_rows(dem_income) |> 
  # bind_rows(dsm) |> 
  bind_rows(past_month_opioid) |> 
  bind_rows(past_month_detox) |> 
  bind_rows(past_month_psych) |> 
  bind_rows(opioid_preferred) |> 
  bind_rows(roa_preferred) |> 
  bind_rows(od) |> 
  mutate(perc = round(perc, 1)) |> 
  rename(` ` = var,
         N = n,
         `%` = perc)

In [ ]:
#| label: tbl-1
#| tbl-cap: "Demographic and Clinical Characteristics"

footnote_table_dem_a <- "N = 301"


table_dem |> 
  knitr::kable() |> 
  kable_classic() |> 
  kableExtra::group_rows(start_row = 2, end_row = 8) |> 
  kableExtra::group_rows(start_row = 10, end_row = 13) |> 
  kableExtra::group_rows(start_row = 15, end_row = 19) |> 
  kableExtra::group_rows(start_row = 21, end_row = 27) |> 
  kableExtra::group_rows(start_row = 29, end_row = 34) |> 
  kableExtra::group_rows(start_row = 36, end_row = 41) |> 
  kableExtra::group_rows(start_row = 43, end_row = 50) |> 
  kableExtra::group_rows(start_row = 52, end_row = 53) |> 
  kableExtra::group_rows(start_row = 55, end_row = 56) |> 
  kableExtra::group_rows(start_row = 58, end_row = 59) |> 
  kableExtra::group_rows(start_row = 61, end_row = 64) |> 
  kableExtra::group_rows(start_row = 66, end_row = 70) |> 
  kableExtra::group_rows(start_row = 72, end_row = 76) |> 
  kableExtra::footnote(general = c(footnote_table_dem_a), escape=FALSE)

N 
 % 
 
 
 
 
 Age 
 
 
 
 
 
 18-21 
 2 
 0.7 
 
 
 22-25 
 7 
 2.3 
 
 
 26-35 
 108 
 35.9 
 
 
 36-45 
 107 
 35.5 
 
 
 46-55 
 60 
 19.9 
 
 
 56-65 
 13 
 4.3 
 
 
 Over 65 
 4 
 1.3 
 
 
 Gender 
 
 
 
 
 
 Man 
 160 
 53.2 
 
 
 Woman 
 134 
 44.5 
 
 
 Non-binary 
 4 
 1.3 
 
 
 Not listed above 
 2 
 0.7 
 
 
 Orientation 
 
 
 
 
 
 Straight, that is, not gay or lesbian 
 238 
 79.1 
 
 
 Lesbian or gay 
 16 
 5.3 
 
 
 Bisexual 
 40 
 13.3 
 
 
 Not sure 
 1 
 0.3 
 
 
 Not listed above 
 4 
 1.3 
 
 
 Race and Ethnicity (select all that apply) 
 
 
 
 
 
 American Indian/Alaskan Native 
 17 
 5.6 
 
 
 Asian 
 4 
 1.3 
 
 
 Black/African American 
 46 
 15.3 
 
 
 Native Hawaiian/Other Pacific Islander 
 3 
 1.0 
 
 
 White/Caucasian 
 237 
 78.7 
 
 
 Hispanic, Latino, or Spanish origin 
 24 
 8.0 
 
 
 Not listed above 
 5 
 1.7 
 
 
 Education 
 
 
 
 
 
 8th grade or less 
 6 
 2.0 
 
 
 Some high school, but did not graduate 
 25 
 8.3 
 
 
 High school graduate or GED 
 95 
 31.6 
 
 
 Some college or 2-year degree 
 131 
 43.5 
 
 
 4-year college graduate 
 30 
 10.0 
 
 
 More than 4-year or advanced degree 
 12 
 4.0 
 
 
 Employment 
 
 
 
 
 
 Disabled, not able to work 
 33 
 11.0 
 
 
 Employed, working 1-39 hours per week 
 66 
 21.9 
 
 
 Employed, working 40 or more hours per week 
 56 
 18.6 
 
 
 Not employed, NOT looking for work 
 31 
 10.3 
 
 
 Not employed, looking for work 
 110 
 36.5 
 
 
 Retired 
 3 
 1.0 
 
 
 Household Income 
 
 
 
 
 
 Less than $25,000 
 170 
 56.5 
 
 
 $25,000 - $34,999 
 41 
 13.6 
 
 
 $35,000 - $49,999 
 38 
 12.6 
 
 
 $50,000 - $74,999 
 28 
 9.3 
 
 
 $75,000 - $99,999 
 13 
 4.3 
 
 
 $100,000 - $149,999 
 4 
 1.3 
 
 
 $150,000 - $199,999 
 4 
 1.3 
 
 
 $200,000 or more 
 1 
 0.3 
 
 
 Past Month Opioid Use 
 
 
 
 
 
 No 
 202 
 67.1 
 
 
 Yes 
 97 
 32.2 
 
 
 Past Month Detox or Residential Treatment 
 
 
 
 
 
 No 
 195 
 64.8 
 
 
 Yes 
 105 
 34.9 
 
 
 Past Month Psychiatric Medication 
 
 
 
 
 
 No 
 154 
 51.2 
 
 
 Yes 
 146 
 48.5 
 
 
 Preferred Opioid 
 
 
 
 
 
 Fentanyl 
 53 
 17.6 
 
 
 Heroin 
 120 
 39.9 
 
 
 Prescription opioid not for opioid treatment 
 96 
 31.9 
 
 
 Medication for opioid treatment 
 29 
 9.6 
 
 
 Preferred Route of Administration 
 
 
 
 
 
 Injection 
 103 
 34.2 
 
 
 Oral 
 57 
 18.9 
 
 
 Smoke 
 54 
 17.9 
 
 
 Sniff or snort 
 83 
 27.6 
 
 
 Other 
 2 
 0.7 
 
 
 Lifetime History of Overdose 
 
 
 
 
 
 Never 
 126 
 41.9 
 
 
 1 time 
 39 
 13.0 
 
 
 2-3 times 
 71 
 23.6 
 
 
 4-5 times 
 26 
 8.6 
 
 
 More than 5 times 
 37 
 12.3 
 
 
 
 Note: 
 
 N = 301